# Stage Mix Generator

If you pay any attention to pop music nowadays, you probably have noticed the larger precense of Kpop. Worldwide, Kpop is gaining popularity out of Asia. Most notebly, BTS has had multiple albums in the Billboard World Albums chart since last summer in addition to making the rounds on late night television. Another popular group, Blackpink, performed at Coachella. 


I personally really enjoy Kpop. I'm going to try and defend that possition as I have failed a great many times before, but at least now with it's popularity, I know I'm not alone

So where does a programming project fit into all of this? Well, there's a paticular part of Kpop that reliably occurs and any time there's a reccurance, there's always a cool project.

Essentially, the mainstream Kpop industry promotes their groups in cycles. I'm no expert but here's how I understand it. (maybe just include an exerpt from a good article about it). Groups will first build hype for a new single or album by releasing teaser images and videos. Then, once the song or album is dropped, the fans will start streaming and the group will already have multiple performances lined up. However, these aren't like traditional tours. The groups have to perform their song at well know events like Inkigayo or M!Countdown, which occur regularly as well. Their performances from each of these shows is now almost guaranteed to be uploaded to YouTube.

Given that the correography for the song does not change, the wealth of footage makes for great montages. This has created a whole new community on YouTube where users will create "stage mixes" of the different performances while a group was promoting.

At a certain point, after being a avid comsumer of this content, I thought to myself, these mixes definitely could be done automatically. All you had to do was align the audio and make the cuts. What could possibly be hard about that? I find the answer to my question because the day after I set out to see if I could do it.

It took a while to work through many bugs and bag logic (I swear I'm good at programming in usually), but I eventually succeeded. Meet Stage Mix Generator (SMG) 1.0.

## Algorithm

The basic algorithm for SMG 1.0 is as follows:

1. Retrieve stage videos and audios
2. Retrieve song audio
3. Align stage videos to audios
4. Make cuts
   - Every 5 seconds **or**
   - Match videos
5. Render video
6. Upload to YouTube

The most interesting parts of the algorithm come with aligning the stage videos with the song audio and making the cuts to match the videos.

## Aligning Stage Videos to Song Audio

## Cutting Mix to Match Video Clip Cuts

## Uploading to YouTube

## Usage

## Conclusion

- conclusion is that programming can be applied to almost anything
    - be creative
    - CS can combine with almost any of your interests